문제 : https://dacon.io/competitions/official/236042/overview/description

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
cd "/content/drive/MyDrive/personal/handwriting_classification"
!unzip -qq "/content/drive/MyDrive/personal/handwriting_classification/open.zip"
'''

'\ncd "/content/drive/MyDrive/personal/handwriting_classification"\n!unzip -qq "/content/drive/MyDrive/personal/handwriting_classification/open.zip"\n'

# import


In [ ]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision.models import resnet18
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

# main

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
CFG = {
    'IMG_HEIGHT_SIZE':64,
    'IMG_WIDTH_SIZE':224,
    'EPOCHS':50,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':256,
    'NUM_WORKERS':2, 
    'SEED':41
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/personal/handwriting_classification/train.csv')


In [ ]:
for i in range(len(df)):
  df.loc[i, 'img_path'] = "/content/drive/MyDrive/personal/handwriting_classification" + df.loc[i,'img_path'][1:]

In [ ]:
df

,id,img_path,label
0,TRAIN_00000,/content/drive/MyDrive/personal/handwriting_cl...,빨간색
1,TRAIN_00001,/content/drive/MyDrive/personal/handwriting_cl...,머
2,TRAIN_00002,/content/drive/MyDrive/personal/handwriting_cl...,차차
3,TRAIN_00003,/content/drive/MyDrive/personal/handwriting_cl...,써
4,TRAIN_00004,/content/drive/MyDrive/personal/handwriting_cl...,놓치다
...,...,...,...
76883,TRAIN_76883,/content/drive/MyDrive/personal/handwriting_cl...,회
76884,TRAIN_76884,/content/drive/MyDrive/personal/handwriting_cl...,겪다
76885,TRAIN_76885,/content/drive/MyDrive/personal/handwriting_cl...,벨트
76886,TRAIN_76886,/content/drive/MyDrive/personal/handwriting_cl...,톼


In [ ]:
# 제공된 학습데이터 중 1글자 샘플들의 단어사전이 학습/테스트 데이터의 모든 글자를 담고 있으므로 학습 데이터로 우선 배치
df['len'] = df['label'].str.len()
train_v1 = df[df['len']==1]

# 제공된 학습데이터 중 2글자 이상의 샘플들에 대해서 단어길이를 고려하여 Train (80%) / Validation (20%) 분할
df = df[df['len']>1]
train_v2, val, _, _ = train_test_split(df, df['len'], test_size=0.2, random_state=CFG['SEED'])

# 학습 데이터로 우선 배치한 1글자 샘플들과 분할된 2글자 이상의 학습 샘플을 concat하여 최종 학습 데이터로 사용
train = pd.concat([train_v1, train_v2])
print(len(train), len(val))

66251 10637


1글자 -> 모두 trainset 

2글자 -> 80% sms train_set, 20%는 valid_set


In [ ]:
# 학습 데이터로부터 단어 사전(Vocabulary) 구축
train_gt = [gt for gt in train['label']]
train_gt = "".join(train_gt)
letters = sorted(list(set(list(train_gt))))
print(letters)
print(len(letters))

vocabulary = ["-"] + letters
print(vocabulary)
print(len(vocabulary))
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
print(idx2char)
char2idx = {v:k for k,v in idx2char.items()}
print(char2idx)

In [ ]:
train.columns

Index(['id', 'img_path', 'label', 'len'], dtype='object')

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.train_mode = train_mode
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, index):
        image = Image.open(self.img_path_list[index]).convert('RGB')
        
        if self.train_mode:
            image = self.train_transform(image)
        else:
            image = self.test_transform(image)
            
        if self.label_list is not None:
            text = self.label_list[index]
            return image, text
        else:
            return image
    
    # Image Augmentation
    def train_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)
    
    def test_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, True)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, False)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

In [ ]:
# image_batch, text_batch = iter(train_loader).__next__()
image_batch, text_batch = next(iter(train_loader))
print(image_batch.size(), text_batch)

torch.Size([256, 3, 64, 224]) ('언덕', '강사', '리듬', '섟', '뻘', '꽉', '도시', '주장', '싱싱하다', '피망', '샨', '와', '뒤집다', '곁', '지역', '뻗다', '론', '희생하다', '한반도', '냑', '공연장', '짜다', '화제', '기성세대', '시리즈', '평', '힘들다', '수', '밖', '온', '의식', '언', '질', '활발히', '탑', '병실', '이별', '자극하다', '확립하다', '와', '왼손', '단어', '등', '감동적', '성숙하다', '뼈', '어리다', '전통적', '양념', '섬', '추천', '밀리미터', '발바닥', '켯', '딴', '마시다', '폄', '하천', '늘', '대사관', '뵙', '적응하다', '찼', '넷', '벌', '락', '늙다', '적어지다', '볶', '극', '송이', '늘', '문자', '열정', '통', '괴로워하다', '반성하다', '함께하다', '임신', '빗방울', '터', '거꾸로', '다방', '삶다', '진단하다', '부인', '오다', '니', '암시', '덴', '스케줄', '자', '톈', '차', '자전거', '웜', '복', '요', '이날', '실감', '좋아지다', '지켜보다', '길이', '깥', '균', '죽다', '얀', '모두', '우울하다', '군', '폭력', '불', '양옆', '공중전화', '계산기', '주무시다', '이해하다', '추측', '임금', '짜증스럽다', '잘못', '자극', '수', '똑바로', '해외여행', '단체', '렐', '걸치다', '오', '대', '취업', '빼앗다', '떱', '쉑', '믄', '포도', '철', '수십', '평', '머리', '고급', '쬡', '현', '깨끗이', '책임', '도움', '단풍', '지나치다', '켈', '입', '손질', '습기', '독하다', '문구', '놓치다', '한참', '겄', '이르다', '닿다', '전부',

In [ ]:
# class RecognitionModel(nn.Module):
#     def __init__(self, num_chars=len(char2idx), rnn_hidden_size=256):
#         super(RecognitionModel, self).__init__()
#         self.num_chars = num_chars
#         self.rnn_hidden_size = rnn_hidden_size
        
#         # CNN Backbone = 사전학습된 resnet18 활용
#         # https://arxiv.org/abs/1512.03385
#         resnet = resnet18(pretrained=True)
#         # CNN Feature Extract
#         resnet_modules = list(resnet.children())[:-3]
#         self.feature_extract = nn.Sequential(
#             *resnet_modules,
#             nn.Conv2d(256, 256, kernel_size=(3,6), stride=1, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(inplace=True)
#         )

#         self.linear1 = nn.Linear(1024, rnn_hidden_size)
        
#         # RNN
#         self.rnn = nn.RNN(input_size=rnn_hidden_size, 
#                             hidden_size=rnn_hidden_size,
#                             bidirectional=True, 
#                             batch_first=True)
#         self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        
        
#     def forward(self, x):
#         # CNN
#         x = self.feature_extract(x) # [batch_size, channels, height, width]
#         x = x.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
         
#         batch_size = x.size(0)
#         T = x.size(1)
#         x = x.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
#         x = self.linear1(x)
        
#         # RNN
#         x, hidden = self.rnn(x)
        
#         output = self.linear2(x)
#         output = output.permute(1, 0, 2) # [T==10, batch_size, num_classes==num_features]
        
#         return output

In [ ]:
class RecognitionModel(nn.Module):
    def __init__(self, num_chars=len(char2idx), rnn_hidden_size=256):
        super(RecognitionModel, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size
        
        # CNN Backbone = 사전학습된 resnet18 활용
        # https://arxiv.org/abs/1512.03385
        resnet = resnet18(pretrained=True)
        # CNN Feature Extract
        resnet_modules = list(resnet.children())[:-3]
        self.feature_extract = nn.Sequential(
            *resnet_modules,
            nn.Conv2d(256, 256, kernel_size=(3,6), stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )

        self.linear1 = nn.Linear(1024, rnn_hidden_size)
        
        # RNN
        self.lstm = nn.LSTM(input_size=rnn_hidden_size, 
                            hidden_size=rnn_hidden_size,
                            bidirectional=True, 
                            batch_first=True)
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        
        
    def forward(self, x):
        # CNN
        x = self.feature_extract(x) # [batch_size, channels, height, width]
        x = x.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
         
        batch_size = x.size(0)
        T = x.size(1)
        x = x.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
        x = self.linear1(x)
        
        # RNN
        x, hidden = self.lstm(x)
        
        output = self.linear2(x)
        output = output.permute(1, 0, 2) # [T==10, batch_size, num_classes==num_features]
        
        return output

In [ ]:
criterion = nn.CTCLoss(blank=0) # idx 0 : '-'


In [ ]:
def encode_text_batch(text_batch):
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.IntTensor(text_batch_targets_lens)
    
    text_batch_concat = "".join(text_batch)
    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.IntTensor(text_batch_targets)
    
    return text_batch_targets, text_batch_targets_lens

In [ ]:
def compute_loss(text_batch, text_batch_logits):
    """
    text_batch: list of strings of length equal to batch size
    text_batch_logits: Tensor of size([T, batch_size, num_classes])
    """
    text_batch_logps = F.log_softmax(text_batch_logits, 2) # [T, batch_size, num_classes]  
    text_batch_logps_lens = torch.full(size=(text_batch_logps.size(1),), 
                                       fill_value=text_batch_logps.size(0), 
                                       dtype=torch.int32).to(device) # [batch_size] 

    text_batch_targets, text_batch_targets_lens = encode_text_batch(text_batch)
    loss = criterion(text_batch_logps, text_batch_targets, text_batch_logps_lens, text_batch_targets_lens)

    return loss

In [ ]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    best_loss = 999999
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for image_batch, text_batch in tqdm(iter(train_loader)):
            image_batch = image_batch.to(device)
            
            optimizer.zero_grad()
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        _train_loss = np.mean(train_loss)
        
        _val_loss = validation(model, val_loader, device)
        print(f'Epoch : [{epoch}] Train CTC Loss : [{_train_loss:.5f}] Val CTC Loss : [{_val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_loss)
        
        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model
            try:
              torch.save(model,"/content/drive/MyDrive/personal/handwriting_classification/bilstm_model_50.pt")
            except:
              try:
                torch.save(model.state_dict(),"/content/drive/MyDrive/personal/handwriting_classification/bilstm_model_state_dict_50.pt")
              except:
                print("model save failed")

    
    return best_model

In [ ]:
def validation(model, val_loader, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for image_batch, text_batch in tqdm(iter(val_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
            
            val_loss.append(loss.item())
    
    _val_loss = np.mean(val_loss)
    return _val_loss

In [ ]:
model = RecognitionModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [1] Train CTC Loss : [6.84575] Val CTC Loss : [4.70296]


  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [49] Train CTC Loss : [0.00074] Val CTC Loss : [0.11195]


  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [50] Train CTC Loss : [0.00074] Val CTC Loss : [0.11098]
Epoch 00050: reducing learning rate of group 0 to 1.9531e-06.


In [ ]:
infer_model_2 = train(infer_model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [1] Train CTC Loss : [0.00073] Val CTC Loss : [0.11087]


  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [49] Train CTC Loss : [0.00071] Val CTC Loss : [0.11299]


  0%|          | 0/259 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

Epoch : [50] Train CTC Loss : [0.00071] Val CTC Loss : [0.11052]


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/personal/handwriting_classification/test.csv')


In [ ]:
for i in range(len(test)):
  test.loc[i, 'img_path'] = "/content/drive/MyDrive/personal/handwriting_classification" + test.loc[i,'img_path'][1:]

In [ ]:
test

,id,img_path
0,TEST_00000,/content/drive/MyDrive/personal/handwriting_cl...
1,TEST_00001,/content/drive/MyDrive/personal/handwriting_cl...
2,TEST_00002,/content/drive/MyDrive/personal/handwriting_cl...
3,TEST_00003,/content/drive/MyDrive/personal/handwriting_cl...
4,TEST_00004,/content/drive/MyDrive/personal/handwriting_cl...
...,...,...
74116,TEST_74116,/content/drive/MyDrive/personal/handwriting_cl...
74117,TEST_74117,/content/drive/MyDrive/personal/handwriting_cl...
74118,TEST_74118,/content/drive/MyDrive/personal/handwriting_cl...
74119,TEST_74119,/content/drive/MyDrive/personal/handwriting_cl...


In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [ ]:
def decode_predictions(text_batch_logits):
    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for image_batch in tqdm(iter(test_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            
            text_batch_pred = decode_predictions(text_batch_logits.cpu())
            
            preds.extend(text_batch_pred)
    return preds

In [ ]:
predictions = inference(infer_model, test_loader, device)


  0%|          | 0/290 [00:00<?, ?it/s]

In [ ]:
predictions_2 = inference(infer_model_2, test_loader, device)

  0%|          | 0/290 [00:00<?, ?it/s]

In [ ]:
# 샘플 별 추론결과를 독립적으로 후처리
def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/personal/handwriting_classification/sample_submission.csv')
submit['label'] = predictions
submit['label'] = submit['label'].apply(correct_prediction)

In [ ]:
submit2 = pd.read_csv('/content/drive/MyDrive/personal/handwriting_classification/sample_submission.csv')
submit2['label'] = predictions_2
submit2['label'] = submit2['label'].apply(correct_prediction)

In [ ]:
submit.to_csv('/content/drive/MyDrive/personal/handwriting_classification/submission_base_lstm_50.csv', index=False)

In [ ]:
submit2.to_csv('/content/drive/MyDrive/personal/handwriting_classification/submission_base_lstm_100.csv', index=False)

In [ ]:
predictions_2

['날---------말',
 '상---------향',
 '밤---이-들--이다',
 '바--------구니',
 '살----------',
 '빼--------놓다',
 '인----식---하다',
 '선---------터',
 '소---------풍',
 '광---------주',
 '나---------나',
 '위---------험',
 '도---------도',
 '슬---------풍',
 '괴--로--워--하다',
 '카---------드',
 '합--------치다',
 '다----경---하다',
 '혼---------자',
 '가----능---하다',
 '호---------주',
 '발----전---되다',
 '피--------우다',
 '스--------웨터',
 '시----작---되다',
 '겨---------울',
 '예---------선',
 '한--------국말',
 '세----워---지다',
 '비--------범밥',
 '좋---------다',
 '남---대-문--시장',
 '보--------수적',
 '사--------진기',
 '내--------리다',
 '평---------핑',
 '맛--------없다',
 '특--------별히',
 '우---------선',
 '대---------답',
 '학---------생',
 '여---------립',
 '본---------질',
 '현---------대',
 '속--------하다',
 '지---저-문--하다',
 '불---------다',
 '아---------래',
 '걸----어---오다',
 '선---------원',
 '호---------주',
 '예----------',
 '골--------프장',
 '가---------위',
 '기---------차',
 '저---------것',
 '기---------침',
 '위---------쪽',
 '불---------만',
 '바---람-직--하다',
 '작----아---지다',
 '학---------비',
 '양-----